# Regressão Logística & Gradiente Descendente - Grupo de DataScience com Python

<p align="center">
  <img alt="decision_tree" src="https://img.devrant.com/devrant/rant/r_1587555_kpz6c.jpg" alt="drawing" width="400" height="400"/>
</p>

#### Autor: Rafael Barbosa - [LinkedIn](https://www.linkedin.com/in/rafael-barbosa0/) - [Github](https://github.com/barbosarafael)

#### **Nota**: Esse **tutorial, post, notebook ou o que_você_quiser_chamar** possui um viés acadêmico (juro que estou tentando perder ele), então tem coisas que provavelmente passarei despercebido em relação aos conceitos. 

#### **Dica**: Se não entendeu algo, mande mensagem ou dê a famosa googlada.



## 1. Gradiente Descendente

<p align="center">
  <img alt="decision_tree" src="https://miro.medium.com/max/875/1*wsBakfF2Geh1zgY4HJbwFQ.gif" alt="drawing" width="800" height="300"/>

  **Exemplo de gradiente descendente**. Retirado de: https://medium.com/@bruno.dorneles/regress%C3%A3o-linear-com-gradiente-descendente-d3420b0b0ff
</p>

- Assim como o OLS, é um método/algoritmo que busca minimizar uma função custo, só que dessa vez **iterativamente**

- Observando o primeiro gráfico do GIF, a parte mais "funda" da nossa curva 3D é chamado de mínimo global, que é onde possui o valor dos parâmetros que minimiza a função de custo

- Começa com um chute inicial para os parâmetros a se estimar ($ \beta $) e, a partir das iterações, estes valores para os parâmetros vão sendo otimizados

- A diferença entre cada iteração é dada por uma taxa de aprendizado $ \alpha $, também chamada de passos

- Se esse $ \alpha $ for grande, então o método irá iteragir poucas vezes podendo não achar um mínimo global

- Se o $ \alpha $ for um valor pequeno demais, o método irá dar pequenos passos, indicando que haverá um número de altas de iterações



## 2. Regressão Logística

### 2.1. Modelo

**Hair et al. (2009)** nos mostram que a regressão logística é utilizada para **modelar** (buscar entender a relação entre as variáveis) e **prever** dados futuros. É utilizada quando a nossa variável independente (y, target, etc.) é binária (justamente nosso caso, já que queremos prever quem sobreviveu ou não no desastre do Titanic) e possuímos uma ou mais variáveis dependentes (explicativas). Estas variáveis explicativas podem ser de natureza binária, categórica (nominal e ordinal) ou contínuas.

A função logística é dada por

$$ P(x) = \frac{exp(\beta_0 + \sum_{k = 1}^{p}\beta_k X_k)}{1 + exp(\beta_0 + \sum_{k = 1}^{p}\beta_k X_k)} $$ 

E tem a seguinte distribuição:
![](https://upload.wikimedia.org/wikipedia/commons/thumb/8/88/Logistic-curve.svg/600px-Logistic-curve.svg.png)

- Dado que os Betas (0 e k) serão estimados pelo método da máxima verossimilhança (ver mais neste [material da ESALQ](http://cmq.esalq.usp.br/BIE5781/lib/exe/fetch.php?media=leituras:verossim.pdf));

- $ X_k $ são as variáveis dependentes (features) que irão para o modelo.
 

### 2.2. Modelagem

**Para modelar**, isto é, verificar a relação entre variáveis iremos utilizar a *Odds Ratio*. Ela é a razão entre a chance de um grupo exposto a uma determinada característica pela chance de um grupo que não foi exposto também apresentar a mesma ocorrência. É dada por

$$\text{OR} = \frac{chance_1}{chance_0}$$ 

### Teste de Wald para os coeficientes

Para verificar quais variáveis são significantes para a modelagem e fazer a interpretação a partir da *Odds Ratio*, os coeficientes de cada variável utilizada no modelo precisam ser significativos. As hipóteses para este teste, para cada variável, são:

$ H_0: \beta_k = 0 $

$ H_1: \beta_k \neq 0 $

Neste caso, iremos utilizar o nível de significância de 5%. Podendo chegar a seguinte regra:

> Se o **p-valor** do teste $ \geq $ 0.05: A variável pode ser utilizada para fazermos a modelagem (logo, esta é significante no modelo);

> Se **p-valor** do teste < 0.05: A variável não pode ser utilizada para fazermos a modelagem (logo, esta não é significante no modelo).

## Aplicação na base de dados do ENEM

In [9]:
#--- Bibliotecas

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

#--- Configurações adicionais

plt.style.use("seaborn-muted")
%matplotlib inline
pd.set_option("display.max_columns", None)

#--- Importando os dados

dados = pd.read_csv(filepath_or_buffer = "data/train.csv")

In [10]:
dados.head()

,Unnamed: 0,NU_INSCRICAO,NU_ANO,CO_MUNICIPIO_RESIDENCIA,NO_MUNICIPIO_RESIDENCIA,CO_UF_RESIDENCIA,SG_UF_RESIDENCIA,NU_IDADE,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_NACIONALIDADE,CO_MUNICIPIO_NASCIMENTO,NO_MUNICIPIO_NASCIMENTO,CO_UF_NASCIMENTO,SG_UF_NASCIMENTO,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,TP_ESCOLA,TP_ENSINO,IN_TREINEIRO,CO_ESCOLA,CO_MUNICIPIO_ESC,NO_MUNICIPIO_ESC,CO_UF_ESC,SG_UF_ESC,TP_DEPENDENCIA_ADM_ESC,TP_LOCALIZACAO_ESC,TP_SIT_FUNC_ESC,IN_BAIXA_VISAO,IN_CEGUEIRA,IN_SURDEZ,IN_DEFICIENCIA_AUDITIVA,IN_SURDO_CEGUEIRA,IN_DEFICIENCIA_FISICA,IN_DEFICIENCIA_MENTAL,IN_DEFICIT_ATENCAO,IN_DISLEXIA,IN_DISCALCULIA,IN_AUTISMO,IN_VISAO_MONOCULAR,IN_OUTRA_DEF,IN_SABATISTA,IN_GESTANTE,IN_LACTANTE,IN_IDOSO,IN_ESTUDA_CLASSE_HOSPITALAR,IN_SEM_RECURSO,IN_BRAILLE,IN_AMPLIADA_24,IN_AMPLIADA_18,IN_LEDOR,IN_ACESSO,IN_TRANSCRICAO,IN_LIBRAS,IN_LEITURA_LABIAL,IN_MESA_CADEIRA_RODAS,IN_MESA_CADEIRA_SEPARADA,IN_APOIO_PERNA,IN_GUIA_INTERPRETE,IN_MACA,IN_COMPUTADOR,IN_CADEIRA_ESPECIAL,IN_CADEIRA_CANHOTO,IN_CADEIRA_ACOLCHOADA,IN_PROVA_DEITADO,IN_MOBILIARIO_OBESO,IN_LAMINA_OVERLAY,IN_PROTETOR_AURICULAR,IN_MEDIDOR_GLICOSE,IN_MAQUINA_BRAILE,IN_SOROBAN,IN_MARCA_PASSO,IN_SONDA,IN_MEDICAMENTOS,IN_SALA_INDIVIDUAL,IN_SALA_ESPECIAL,IN_SALA_ACOMPANHANTE,IN_MOBILIARIO_ESPECIFICO,IN_MATERIAL_ESPECIFICO,IN_NOME_SOCIAL,IN_CERTIFICADO,NO_ENTIDADE_CERTIFICACAO,CO_UF_ENTIDADE_CERTIFICACAO,SG_UF_ENTIDADE_CERTIFICACAO,CO_MUNICIPIO_PROVA,NO_MUNICIPIO_PROVA,CO_UF_PROVA,SG_UF_PROVA,TP_PRESENCA_CN,TP_PRESENCA_CH,TP_PRESENCA_LC,TP_PRESENCA_MT,CO_PROVA_CN,CO_PROVA_CH,CO_PROVA_LC,CO_PROVA_MT,NU_NOTA_CN,NU_NOTA_CH,NU_NOTA_LC,NU_NOTA_MT,TX_RESPOSTAS_CN,TX_RESPOSTAS_CH,TX_RESPOSTAS_LC,TX_RESPOSTAS_MT,TP_LINGUA,TX_GABARITO_CN,TX_GABARITO_CH,TX_GABARITO_LC,TX_GABARITO_MT,TP_STATUS_REDACAO,NU_NOTA_COMP1,NU_NOTA_COMP2,NU_NOTA_COMP3,NU_NOTA_COMP4,NU_NOTA_COMP5,NU_NOTA_REDACAO,Q001,Q002,Q003,Q004,Q005,Q006,Q007,Q008,Q009,Q010,Q011,Q012,Q013,Q014,Q015,Q016,Q017,Q018,Q019,Q020,Q021,Q022,Q023,Q024,Q025,Q026,Q027,Q028,Q029,Q030,Q031,Q032,Q033,Q034,Q035,Q036,Q037,Q038,Q039,Q040,Q041,Q042,Q043,Q044,Q045,Q046,Q047,Q048,Q049,Q050
0,1,ed50e8aaa58e7a806c337585efee9ca41f1eb1ad,2016,4314902,Porto Alegre,43,RS,24,M,0.0,1,1,4314902.0,Porto Alegre,43.0,RS,1,4,1,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,4314902,Porto Alegre,43,RS,1,1,1,1,16f84b7b3d2aeaff7d2f01297e6b3d0e25c77bb2,9cd70f1b922e02bd33453b3f607f5a644fb9b1b8,01af53cd161a420fff1767129c10de560cc264dd,97caab1e1533dba217deb7ef41490f52e459ab01,436.3,495.4,581.2,399.4,ABACCABDBEDEDEEDDDBBBDAABABECDCCCAAABCBEEABCA,BBEBCAEBCEDBDDEBCACDBCBDABDDCDBEDCEAEABCAADAC,99999DCABCABCBBCADBCBBABAEADBEEBEDAAABAEAECCBB...,AECCCBABDCADDBCBCCADADBBAADEBCEEBABCDDEAABCAA,1,BCADBCEDCCBADBDCADBEBDBEADCCAEACDBEDBCACAEBEE,BCBCCADCCBEDDBEEADBDCECCBBECAEEDDADCDDACAADBA,CEDEACCBEBDBABECABDADEADCEDCBEBBCEBAAEBAAECBBC...,DADCCECBDECADCBACDBCEEABDDAABDBCEADDBECEBAEBC,1.0,120.0,120.0,120.0,80.0,80.0,520.0,D,D,B,B,3,C,A,B,B,B,A,B,A,A,A,A,A,A,B,A,A,C,A,A,A,C,H,E,5.0,5.0,1.0,0.0,0.0,5,5,0,5,5,5,5,5.0,A,A,A,A,A,A,A,B,D
1,2,2c3acac4b33ec2b195d77e7c04a2d75727fad723,2016,2304707,Granja,23,CE,17,F,0.0,3,1,2304707.0,Granja,23.0,CE,2,0,2,1.0,0,23005157.0,2304707.0,Granja,23.0,CE,2.0,1.0,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,2304707,Granja,23,CE,1,1,1,1,b9b06ce8c319a3df2158ea3d0aef0f7d3eecaed7,909237ab0d84688e10c0470e2997348aff585273,01af53cd161a420fff1767129c10de560cc264dd,97caab1e1533dba217deb7ef41490f52e459ab01,474.5,544.1,599.0,459.8,EDAAABDABADBCCAEDCDDECABADBEECBBAEEDCABCAABBC,ADCAABAADCEDAACDACEEDACBDCBACEDEDBACAADADABDC,99999ACBACDBDBCEADDAEEADCCEACEEDBECAAAACBECDAC...,AEDACBDECEDAEEAEDDEBDECDCECACADCACACAABDACEAB,1,EBEECADBCABECDDBEADCCABDEABEBCADBADDCCCEDBBCA,DCECBAAADCBBECAEDCEDDACCBDDACEDDBADBCADBCBCEE,CEDEACCBEBDBABECABDADEADCEDCBEBBCEBAAEBAAECBBC...,DADCCECBDECADCBACDBCEEABDDAABDBCEA

In [13]:
dados.shape

(13730, 167)